# Project 02: One ligand against several targets

This notebook shows how plipify can be used to compare the binding modes of a small molecule towards multiple protein. This can be especially interesting for protein families with selectivity issues, for example kinases.

**Workflow:**  
01. Retrieve data from KLIFS  
02. Analyze data with plipify  
03. Generate publication ready figures  

## 01. Retrieve data from KLIFS

In [1]:
from tqdm.auto import tqdm
from opencadd.databases.klifs import setup_remote

INFO:opencadd.databases.klifs.api:If you want to see an non-truncated version of the DataFrames in this module, use `pd.set_option('display.max_columns', 50)` in your notebook.


In [2]:
# Set up remote session
klifs = setup_remote()

INFO:opencadd.databases.klifs.api:Set up remote session...
INFO:opencadd.databases.klifs.api:Remote session is ready!


In [3]:
# retrieve kinase structures
kinase_df = klifs.structures.all_structures()

In [4]:
# remove structures without orthosteric ligand
kinase_df = kinase_df[kinase_df["ligand.expo_id"] != "-"]

In [5]:
# sort by quality
kinase_df = kinase_df.sort_values(by=["structure.qualityscore", "structure.resolution"],ascending=[False, True])

In [6]:
# find unique kinase ligand pairs
kinase_ligand_pairs_df = kinase_df.groupby(["kinase.klifs_id", "ligand.expo_id"]).head(1)

In [7]:
# find ligands binding to many kinases
topligands = kinase_ligand_pairs_df.groupby("ligand.expo_id").size().sort_values(ascending=False)
topligands.head(10)

ligand.expo_id
ANP    76
ADP    51
STU    40
ATP    32
ACP    19
AGS    17
1N1    14
STI    10
AMP    10
DB8    10
dtype: int64

More details for these ligands:

In [8]:
import html
ligand_details = klifs.ligands.by_ligand_expo_id(topligands.index[:10].tolist())
ligand_details["ligand.name"] = ligand_details["ligand.name"].apply(html.unescape)
ligand_details

,ligand.klifs_id,ligand.expo_id,ligand.name,ligand.smiles,ligand.inchikey
0,26,ADP,ADENOSINE-5'-DIPHOSPHATE,P(=O)(OP(=O)(O)O)(OC[C@H]1O[C@@H](N2c3ncnc(N)c...,XTWYTFMLZFPYCI-KQYNXXCUSA-N
1,28,STU,STAUROSPORINE,O=C1NCc2c3c4N(c5c3cccc5)[C@]6(O[C@@H](N7c4c(c8...,HKSZLNNOFSGOKW-FYTWVXJKSA-N
2,38,AGS,PHOSPHOTHIOPHOSPHORIC ACID-ADENYLATE ESTER,S=P(OP(=O)(OP(=O)(OC[C@H]1O[C@@H](N2c3ncnc(N)c...,NLTUCYMLOPLUHL-KQYNXXCUSA-N
3,48,STI,4-(4-METHYL-PIPERAZIN-1-YLMETHYL)-N-[4-METHYL-...,O=C(Nc1cc(Nc2nc(c3cnccc3)ccn2)c(cc1)C)c4ccc(cc...,KTUFNOKKBVMGRW-UHFFFAOYSA-N
4,63,ATP,ADENOSINE-5'-TRIPHOSPHATE,P(=O)(OP(=O)(OC[C@H]1O[C@@H](N2c3ncnc(N)c3N=C2...,ZKHQWZAMYRWXGA-KQYNXXCUSA-N
5,64,ANP,PHOSPHOAMINOPHOSPHONIC ACID-ADENYLATE ESTER,P(=O)(OP(=O)(O)NP(=O)(O)O)(OC[C@H]1O[C@@H](N2c...,PVKSNHVPLWYQGJ-KQYNXXCUSA-N
6,140,AMP,ADENOSINE MONOPHOSPHATE,P(=O)(OC[C@H]1O[C@@H](N2c3ncnc(N)c3N=C2)[C@H](...,UDMBCSSLTHHNCD-KQYNXXCUSA-N
7,160,ACP,PHOSPHOMETHYLPHOSPHONIC ACID ADENYLATE ESTER,P(=O)(OP(=O)(O)CP(=O)(O)O)(OC[C@H]1O[C@@H](N2c...,UFZTZBNSLXELAL-IOSLPCCCSA-N
8,252,DB8,"4-[(2,4-dichloro-5-methoxyphenyl)amino]-6-meth...",Clc1c(OC)cc(Nc2c(C#N)cnc3c2cc(OC)c(OCCCN4CCN(C...,UBPYILGKFZZVDX-UHFFFAOYSA-N
9,281,1N1,N-(2-CHLORO-6-METHYLPHENYL)-2-({6-[4-(2-HYDROX...,Clc1c(NC(=O)C=2SC(=NC2)Nc3nc(nc(N4CCN(CCO)CC4)...,ZBNZXTGUTAYRHI-UHFFFAOYSA-N


### Get PDB files

In [9]:
from pathlib import Path

LIGAND = "STU"  # any ligand expo id, see above for relevant options

HERE = Path(_dh[-1])
DATA = HERE / "data" / LIGAND
OUT = HERE / "output" / LIGAND
DATA.mkdir(exist_ok=True, parents=True)
OUT.mkdir(exist_ok=True, parents=True)

Retrieve all Human structures for ligand=${LIGAND}, and keep the first result for each kinase ID (we don't want to bias the interactions for the most popular kinases). We need a 2nd query to get the families and groups in case we want to cluster results later.

In [10]:
structures_for_ligand = (
    klifs.structures.by_ligand_expo_id(LIGAND)
    .query("`species.klifs` == 'Human'")
    .drop_duplicates(subset=["kinase.klifs_id"])
)
families_and_groups = klifs.kinases.by_kinase_klifs_id(structures_for_ligand["kinase.klifs_id"].tolist())
structures_for_ligand["kinase.family"] = families_and_groups["kinase.family"].tolist()
structures_for_ligand["kinase.group"] = families_and_groups["kinase.group"].tolist()

structures_for_ligand

,structure.klifs_id,structure.pdb_id,structure.alternate_model,structure.chain,species.klifs,kinase.klifs_id,kinase.klifs_name,kinase.family,kinase.group,structure.pocket,...,structure.bp_ii_b_in,structure.bp_ii_out,structure.bp_ii_b,structure.bp_iii,structure.bp_iv,structure.bp_v,structure.grich_distance,structure.grich_angle,structure.grich_rotation,structure.filepath
0,7086,5loh,A,A,Human,22,MASTL,MAST,AGC,KPIS____GKVYLYAVKVV________LALSKPFIVHLYYSYLVME...,...,False,False,False,False,False,False,0.0000,0.0000,0.0000,None
4,3164,1oky,A,A,Human,27,PDK1,PDK1,AGC,KILGEGSFSTVVLYAIKILYVTRERDVMSRLDPFFVKLYFTYFGLS...,...,False,False,False,False,False,False,17.0610,57.3762,55.5831,None
6,5357,1xjd,-,A,Human,37,PKCt,PKC,AGC,KMLGKGSFGKVFLFAIKALCTMVEKRVLSLAWPFLTHMFCTFFVME...,...,False,False,False,False,False,False,15.2886,50.6519,78.0833,None
7,1856,3a60,-,B,Human,51,p70S6K,RSK,AGC,RVLGKGGYGKVFQFAMKVL_____________PFIVDLIYAYLILE...,...,False,False,False,False,False,False,15.4728,50.0971,69.5272,None
12,2777,2z7r,-,A,Human,54,RSK1,RSK,AGC,KVLGQGSFGKVFLYAMKVL_______ILADVNPFVVKLHYAYLILD...,...,False,False,False,False,False,False,16.0725,54.4858,50.9929,None
13,4686,4fr4,B,F,Human,68,YANK1,YANK,AGC,RAIGKGSFGKVCIYAMKYMNVFKELQIMQGLEPFLVNLWYSFMVVD...,...,False,False,False,False,False,False,19.0060,64.9302,69.5296,None
25,121,4rer,-,A,Human,117,AMPKa1,CAMKL,CAMK,DTLGVGTFGKVKVVAVKILKIRREIQNLKLFRPHIIKLYQVFMVME...,...,False,False,False,False,False,False,16.1752,54.9349,64.2654,None
31,190,4cff,-,C,Human,118,AMPKa2,CAMKL,CAMK,DTLGVGTFGKVKIVAVKILKIKREIQNLKLFRPHIIKLYQVFMVME...,...,False,False,False,False,False,False,15.0069,51.1153,61.1399,None
46,8603,5oor,-,A,Human,121,CHK1,CAMKL,CAMK,QTLGEGAYGEVQLVAVKIVNIKKEICILKMLNENVIKFYGHYLFME...,...,False,False,False,False,False,False,17.0900,58.5771,60.6049,None
48,5661,1wvy,-,A,Human,140,DAPK1,DAPK,CAMK,EELGSGQFAVVKKYAAKFIDIEREVSILKEIQPNVITLHEVILILE...,...,False,False,False,False,False,False,16.1416,54.7014,47.6587,None


In [11]:
if not list(DATA.glob("*")):  # if dir is empty, download
    paths = []
    for row in tqdm(structures_for_ligand.to_dict("records")):
        outdir = DATA / f'{row["structure.pdb_id"]}_{row["structure.klifs_id"]}'
        path = klifs.coordinates.to_pdb(row["structure.klifs_id"], outdir)
        klifs.coordinates.to_mol2(row["structure.klifs_id"], outdir)
        paths.append(path)
    structures_for_ligand["structure.filepath"] = paths

In [12]:
pdbs = sorted((DATA).glob("**/*.pdb"))

In [13]:
filename_metadata_by_structure_id = {}
for pdb in pdbs:
    pdb_id, structure_id = pdb.parent.name.split("_")
    chain_id = pdb.stem.split("_chain")[1][0]
    filename_metadata_by_structure_id[int(structure_id)] = {
        "pdb_id": pdb_id,
        "chain_id": chain_id,
        "filename": pdb
    }

__TODO__: We need to convert mol2 to pdb for nglview and plipify compatibility. We could download the PDBs directly, but some residue numbers are wrong, apparently.

Compare the files in the DATA folders.

In [14]:
import nglview as nv

view = nv.NGLWidget()
for pdb in pdbs[:10]:
    structure = nv.adaptor.FileStructure(str(pdb))
    view.add_component(structure)
view.center(LIGAND)
view

NGLWidget()

Get their interaction fingerprints

In [15]:
fingerprints = klifs.interactions.by_structure_klifs_id(structures_for_ligand["structure.klifs_id"].tolist())
fingerprints.head()

,structure.klifs_id,interaction.fingerprint
0,33,0000000000000010000001000000000000000000000000...
1,121,0000000000000010000001000000100000010000000000...
2,190,0000000000000010000001000000100000010000000000...
3,373,0000000000000010000001000000100000010000000000...
4,705,0000000000000010000001000000000000010000000000...


Get their pocket numbering

In [16]:
pockets = []
# FIXME: The whole mess of structure dataframes, PDB files and pocket dataframes need to be 
# tied together in a better way. Right now we are relying on the implicit order provided by
# the sorted PDB ids but this is not guaranteed to be robust if we do not drop the duplicates
# in the preprocessing! Revisit ASAP, using some kind of dictionary on the structure ID or
# something. Chain IDs should also be stored in a cleaner way that does not rely on the 
# filename itself!
for i in structures_for_ligand.sort_values("structure.pdb_id")["structure.klifs_id"].tolist():
    pocket = klifs.pockets.by_structure_klifs_id(i)        
    pocket["chain_id"] = filename_metadata_by_structure_id[i]["chain_id"]
    pocket["structure.klifs_id"] = i
    pockets.append(pocket)

display(pockets[0])

,residue.klifs_id,residue.id,residue.klifs_region_id,residue.klifs_region,residue.klifs_color,chain_id,structure.klifs_id
0,1,8,I.1,I,khaki,A,4267
1,2,9,I.2,I,khaki,A,4267
2,3,10,I.3,I,khaki,A,4267
3,4,11,g.l.4,g.l,green,A,4267
4,5,12,g.l.5,g.l,green,A,4267
...,...,...,...,...,...,...,...
80,81,145,xDFG.81,xDFG,cornflowerblue,A,4267
81,82,146,xDFG.82,xDFG,cornflowerblue,A,4267
82,83,147,xDFG.83,xDFG,cornflowerblue,A,4267
83,84,148,a.l.84,a.l,cornflowerblue,A,4267


## 02. Analyze data with plipify

TBD

In [17]:
from plip.structure.preparation import PDBComplex
pdbcomplex = PDBComplex()
pdbcomplex.load_pdb(str(pdbs[0]))

INFO:plip.structure.preparation:PDB structure successfully read
INFO:plip.structure.preparation:protonated structure written to /tmp/HUMAN_CDK2_1aq1_chainA_complex_protonated.pdb
INFO:plip.structure.preparation:analyzing one ligand


In [18]:
import logging
logger = logging.getLogger("plip")
logger.setLevel(logging.WARNING)

In [19]:
from importlib import reload
import plipify.fingerprints
import plipify.core
reload(plipify.fingerprints)
reload(plipify.core)
from plipify.core import Structure
from plipify.fingerprints import InteractionFingerprint

In [20]:
structures = [Structure.from_pdbfile(str(path), ligand_identifier=LIGAND) for path in tqdm(pdbs)]

In [21]:
# Filter out some unwanted structures
# - only those with a single binding site
# - that corresponds to the ligand we requested
picked_structures = []
picked_pockets = []
# FIXME Structures and pockets are NOT parallel lists!!!
for s, p in zip(structures, pockets):
    if len(s.binding_sites) == 1 and s.binding_sites[0].name.startswith(LIGAND):
        picked_structures.append(s)
        picked_pockets.append(p)

In [22]:
import pandas as pd
from IPython.display import Markdown
for s in picked_structures:
    display(Markdown(f"### {Path(s._path).stem}"))
    for bs in s.binding_sites:
        display(Markdown(f"#### {bs.name}"))
        for name, interactions in bs.interactions.items():
            display(Markdown(f"##### {name}"))
            display(pd.concat([ii.to_dataframe() for ii in interactions]))


### HUMAN_CDK2_1aq1_chainA_complex

#### STU:A:299

##### hydrophobic

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,DIST,LIGCARBONIDX,PROTCARBONIDX,LIGCOO,PROTCOO
0,18,VAL,A,299,STU,A,3.96,2373,141,"(2.916, 16.384, 38.12)","(5.412, 16.187, 41.193)"
0,31,ALA,A,299,STU,A,3.96,2368,242,"(0.172, 19.056, 42.68)","(2.993, 21.123, 44.529)"
0,10,ILE,A,299,STU,A,3.71,2381,87,"(-1.059, 17.163, 44.123)","(2.006, 17.169, 46.21)"
0,33,LYS,A,299,STU,A,3.93,2374,257,"(3.53, 17.334, 37.359)","(7.323, 18.132, 37.996)"
0,80,PHE,A,299,STU,A,3.92,2375,576,"(3.444, 18.718, 37.695)","(5.294, 22.038, 38.647)"
0,134,LEU,A,299,STU,A,3.53,2369,1017,"(0.906, 19.264, 41.618)","(-2.317, 19.802, 40.288)"
0,134,LEU,A,299,STU,A,3.68,2367,1018,"(-0.263, 17.713, 43.101)","(-3.428, 17.746, 41.232)"
0,145,ASP,A,299,STU,A,3.50,2374,1094,"(3.53, 17.334, 37.359)","(1.663, 17.026, 34.41)"


##### hbond-don

...

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,DIST,LIGCARBONIDX,PROTCARBONIDX,LIGCOO,PROTCOO
0,327,ILE,A,601,STU,A,3.60,2341,228,"(-1.249, 17.654, 44.966)","(1.83, 16.884, 46.672)"
0,350,LYS,A,601,STU,A,3.53,2330,380,"(3.655, 19.139, 37.365)","(7.139, 19.541, 37.765)"
0,447,LEU,A,601,STU,A,3.39,2323,1166,"(0.388, 19.161, 42.422)","(-2.214, 19.258, 40.255)"


##### hbond-don

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,SIDECHAIN,DIST_H-A,DIST_D-A,DON_ANGLE,PROTISDON,DONORIDX,DONORTYPE,ACCEPTORIDX,ACCEPTORTYPE,LIGCOO,PROTCOO
0,329,GLU,A,601,STU,A,False,3.77,4.10,102.26,True,233,Nam,2312,O3,"(1.942, 14.307, 41.315)","(4.963, 11.574, 41.741)"
0,398,LEU,A,601,STU,A,False,1.86,2.83,167.40,True,795,Nam,2335,O2,"(-0.68, 20.538, 44.056)","(-2.489, 22.323, 45.303)"


##### hbond-acc

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,SIDECHAIN,DIST_H-A,DIST_D-A,DON_ANGLE,PROTISDON,DONORIDX,DONORTYPE,ACCEPTORIDX,ACCEPTORTYPE,LIGCOO,PROTCOO
0,444,ASP,A,601,STU,A,False,2.18,3.12,153.41,False,2345,N3,1141,O2,"(-0.878, 12.499, 39.029)","(-2.733, 14.086, 37.083)"


__TODO__: The alignment maps are not calculated correctly for these structures. Trying to filter some out it's not working, so I'll need to investigate a bit more.

In [23]:
klifs_mappings = []
for p in picked_pockets:
    mapping = {}
    for index, kid, resid, chain_id in p[["residue.klifs_id", "residue.id", "chain_id"]].itertuples():
        # This mapping key will be the LABEL of the fingerprint
        mapping[f"KLI{kid}"] = {
            # these keys must match signature in plipify.core.Structure.get_residue_by
            "seq_index": int(resid) if resid != "_" else None,
            "chain": chain_id,
        }
    klifs_mappings.append(mapping)

In [24]:
fp = InteractionFingerprint().calculate_fingerprint(
        picked_structures,
        residue_indices=klifs_mappings,
        labeled=True, 
        as_dataframe=True, 
        remove_non_interacting_residues=True,
        remove_empty_interaction_types=True,
        ensure_same_sequence=False
    )

if not fp.values.shape[0]:
    raise ValueError("Fingerprint is empty!")

fp.style.background_gradient(axis=None, cmap="YlGnBu")

! Warning, could not process structure <Structure with 294 residues, 0 ligands and 1 binding sites (loaded from file `/home/jaime/devel/py/volkamerlab/plipify/projects/02/data/STU/1nxk_1312/HUMAN_MAPKAPK2_1nxk_chainC_complex.pdb`)> due to error `ValueError`: No residue with such sequence index: 94, C!
! Warning, could not process structure <Structure with 267 residues, 0 ligands and 1 binding sites (loaded from file `/home/jaime/devel/py/volkamerlab/plipify/projects/02/data/STU/1yhs_2444/HUMAN_PIM1_1yhs_chainA_complex.pdb`)> due to error `ValueError`: No residue with such sequence index: 161, A!


,hydrophobic,hbond-don,hbond-acc,waterbridge,pistacking
KLI3,54,0,0,0,0
KLI4,0,1,0,0,0
KLI5,0,2,0,4,0
KLI6,0,1,0,0,0
KLI10,1,0,0,0,0
KLI11,25,0,0,0,0
KLI12,2,0,0,0,1
KLI15,7,0,0,0,0
KLI17,21,0,0,0,0
KLI36,3,0,0,0,0


## 03. Generate publication ready figures

TBD